# はてなブックマークの記事のurlからhtmlデータを解析する
### htmlデータから取れそうな要素

URL,公開日、タイトル、記事内のリンク数、はてな関連記事のリンク数、はてなブックマーク数、本文、ヘッダー情報、それぞれのhtmlタグの数

In [1]:
# mongodbからcsvファイルを取ってくる
import pandas as pd
import os
def fetchData():
   from pymongo import MongoClient
   if not os.path.exists("./data"):
       os.mkdir("./data")
   url = open('dbserver.txt', 'r').read()
   # MongoDBに接続し, テキストデータを取得
   client = MongoClient(url)
   collection = client["test"]["hateb"]
   df = pd.DataFrame.from_dict(list(collection.find())).astype(object)
   pd.DataFrame.to_csv(df, "./data/hateb.csv")



In [2]:
#data/hateb.csvがない人は#を外して実行
#fetchData()

In [3]:
#coding: utf-8
from bs4 import BeautifulSoup
import requests
from datetime import datetime

import re
from urllib.parse import urlparse
# aタグリストをurlリストに変換
def a_to_href(a_tags):
    hrefs = list()
    for a_tag in a_tags:
        hrefs.append(a_tag.get('href'))
    return hrefs

# はてな関連のurlの数とその他のurlの数をカウント
def count_hrefs(hrefs):
    count = 0
    hatena = 0
    for href in hrefs:
        o = urlparse(href)
        if len(o.scheme) > 0:
            count +=1
            if re.search('hate(blo|na)',href):
                hatena +=1
        else:
            continue
    return {'total':count,'hatena':hatena,'else':count-hatena}

In [4]:
from urllib.parse import urlparse

#urlの先を解析
def analyse_dest(url) :
    
    #エラー処理
    try:
        res = requests.get(url)
        if res.status_code is not 200:
           #print("404 not found")
           return
        
        header = res.headers
        soup = BeautifulSoup(res.text, 'html.parser')
        title = soup.title.text
        text = soup.text
    
        api = 'https://bookmark.hatenaapis.com/count/entry?url={}'.format(url)
        res=requests.get(api)
        bookmark=res.json()
    
        tags_html = soup.find_all('a',{'class':'entry-category-link'})
        tags = list()
        for tag_html in tags_html:
            tags.append(tag_html.text)
    
        links = count_hrefs(a_to_href(soup.find_all('a')))
    
        counts = dict()
        for tag in soup.find_all(True):
            counts[tag.name] = counts.get(tag.name,0)+1
        
        #はてなブログ
        #published_at = soup.select_one('div > header > div.date.entry-date.first > a > time').get('datetime')
        if soup.find('time') is not None:
            published_at = soup.find('time').get('datetime')
        else:
            published_at=None
    
    except Exception as e:
        #print("error:", e.args)
        return 
    
    dic = {'url':url,'published_at':published_at ,'title': title,'bookmark':bookmark,'tags':tags,
          'links':links,'counts':counts,'header':header,'text':text,'html':soup}
    
    return dic

#dataframeを一行ずつ処理する
def analyse_row(row) :
    try:
        soup = BeautifulSoup(row['html'], 'html.parser')
        title = soup.title.text
        text = soup.text
    
        api = 'https://bookmark.hatenaapis.com/count/entry?url={}'.format(row['url'])
        res=requests.get(api)
        bookmark=res.json()
    
        tags_html = soup.find_all('a',{'class':'entry-category-link'})
        tags = list()
        for tag_html in tags_html:
            tags.append(tag_html.text)
    
        links = count_hrefs(a_to_href(soup.find_all('a')))
    
        counts = dict()
        for tag in soup.find_all(True):
            counts[tag.name] = counts.get(tag.name,0)+1
        
        #はてなブログ
        #published_at = soup.select_one('div > header > div.date.entry-date.first > a > time').get('datetime')
        if soup.find('time') is not None:
            published_at = soup.find('time').get('datetime')
        else:
            published_at=None
    
    except Exception as e:
        print("error:", e.args)
        return 
    
    dic = {'published_at':published_at ,'title': title,'bookmark':bookmark,'tags':tags,
          'links':links,'counts':counts,'text':text,'html':soup}
    
    return dic

#### analyse_dest(url)の説明
urlの入力から辞書型データを返す
辞書型のデータは

+ url : URL  
+ published_at : 公開日 
+ title : タイトル 
+ bookmark : はてなブックマーク数
+ tags : 記事にひも付いたタグのリスト
+ links(辞書) 
    +  total : 記事内の合計link数 
    +  hatena : はてな関連記事のlink数 
    + else : はてな以外の記事のlink数 
+ counts(辞書)
    + head: headタグの数 
    + meta: metaタグの数 
    + link :linkタグの数
    + ...
+ header : ヘッダー情報
+ text : 文章
+ html : htmlそのまま 

### analyze_row(row)の説明
htmlデータの入力から辞書型データを返す
辞書型のデータは

+ url : URL  
+ published_at : 公開日 
+ title : タイトル 
+ bookmark : はてなブックマーク数
+ tags : 記事にひも付いたタグのリスト
+ links : (辞書) 
    +  total : 記事内の合計link数 
    +  hatena : はてな関連記事のlink数 
    +  else : はてな以外の記事のlink数 
+ counts : (辞書)
    + head : headタグの数 
    + meta : metaタグの数 
    + link :linkタグの数
    + ...
+ text : 文章
+ html : htmlそのまま 

In [5]:
import requests
#テスト用
#url="http://rootport.hateblo.jp/entry/20110905/1315233706"
#url = "http://plus14.hateblo.jp/entry/2019/10/26/193256"
url ="https://mohritaroh.hateblo.jp/entry/2019/09/14/203000"
#url="https://qiita.com/wanwanland/items/ce272419dde2f95cdabc"
res = requests.get(url)
analyse_dest(url)

{'url': 'https://mohritaroh.hateblo.jp/entry/2019/09/14/203000',
 'published_at': '2019-09-14T11:30:00Z',
 'title': 'みずほ銀行の新システムがIBM×COBOLで昭和っぽさあるとおもったら逆で、みずほだけが「脱・昭和」できてたのか - in between days',
 'bookmark': 644,
 'tags': ['IT'],
 'links': {'total': 45, 'hatena': 36, 'else': 9},
 'counts': {'html': 1,
  'head': 1,
  'meta': 27,
  'title': 1,
  'link': 11,
  'script': 27,
  'style': 1,
  'body': 1,
  'div': 94,
  'a': 52,
  'iframe': 2,
  'header': 2,
  'h1': 2,
  'h2': 1,
  'article': 1,
  'time': 5,
  'span': 31,
  'p': 26,
  'small': 2,
  'cite': 3,
  'img': 7,
  'blockquote': 4,
  'br': 1,
  'b': 4,
  'footer': 2,
  'ul': 4,
  'li': 12,
  'ins': 2,
  'aside': 2,
  'form': 1,
  'input': 2,
  'i': 6,
  'u': 2,
  'h3': 1,
  'address': 1,
  'button': 5},
 'header': {'Server': 'nginx', 'Date': 'Thu, 21 Nov 2019 07:30:59 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding, User-Agent, X-Forwarded-Host, X-Dev

In [6]:
def dic_for_df(result):
    """
    辞書をdataframe変換用の辞書に変換する
    """
   
    try:
        #辞書を値をする辞書とhtmlをpop
        #dic_header = result.pop('header')
        dic_links = result.pop('links')
        dic_counts = result.pop('counts')
        html= result.pop('html')
        
        
        #辞書を値をする辞書の値を追加
        result['link_else']=dic_links['else']
        
        tag_list=['script','div','link','a','span','p','img','li','h1','h2','h3']
        for tag in tag_list:
            if tag in dic_counts:
                result[tag]= dic_counts[tag]
            else:
                result[tag]= 0
    except Exception as e:
        print("error:", e.args)
        return 
    return result

In [7]:
#csvfileの読み込み
import pandas as pd
df_in = pd.read_csv("./data/hateb.csv")

In [9]:
#dataframe変換用に辞書型データのリストを作成
dic_list = list()
# header情報が欲しい場合
# for url in df_in['url']:
#     result = analyse_dest(url)
#     dic_list.append(dic_for_df(result))
for index,row in df_in.iterrows():
    result = analyse_row(row)
    if result is None:
        continue
    
    result['url']=row['url']
    result['category']=row['category']
    dic_list.append(dic_for_df(result))


In [ ]:
#csvfileを作成しローカルに保存
import pandas as pd
import os
df=pd.DataFrame(dic_list)
if not os.path.exists("./data"):
       os.mkdir("./data")
df.to_csv('./data/hateb_reshaped2.csv')

In [ ]:
import sys
import pandas as pd
import pymongo
import json



def import_content(filepath):
    url = open('dbserver.txt', 'r').read()
    mng_client = pymongo.MongoClient(url)
    #mng_client = pymongo.MongoClient('localhost', 27017)
    mng_db = mng_client["test"] # Replace mongo db name
    collection_name = 'hateb_reshaped2' # Replace mongo db collection name
    db_cm = mng_db[collection_name]
    data = pd.read_csv(filepath)
    data_json = json.loads(data.to_json(orient='records'))
    db_cm.remove()
    db_cm.insert(data_json)


In [11]:
filepath = "./data/hateb_reshaped2.csv"

In [12]:
#mongodbにcsvファイルを送信
import_content(filepath)

/Users/ito/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
/Users/ito/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
